<a href="https://colab.research.google.com/github/Vaycold/statistics_seminar/blob/main/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
os.environ['KAGGLE_USERNAME'] = 'kimgodbang'
os.environ['KAGGLE_KEY'] = '1c0e1716b23829d4381dcbced37ba49b'

In [2]:
!rm *.*
!kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

rm: cannot remove '*.*': No such file or directory
  0% 0.00/254k [00:00<?, ?B/s]
100% 254k/254k [00:00<00:00, 38.2MB/s]
  0% 0.00/518k [00:00<?, ?B/s]
100% 518k/518k [00:00<00:00, 74.0MB/s]


In [3]:
df = pd.read_csv('training.csv')

In [4]:
df.shape

(5000, 3)

In [5]:
onehot = np.zeros((5000,11))
onehot.shape

(5000, 11)

In [6]:
def labelsplit(labels) :
    
    label = labels.replace(' ',',').split(',')
    for idx, lab in enumerate(label) :
        label[idx] = int(lab)
    return label

In [7]:
df['label']  = df['Labels'].apply(lambda x :  labelsplit(x))

In [8]:
df.head()

,ID,Tweet,Labels,label
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10,"[0, 10]"
1,2,BanMediaHouse whose is responsible for spreadi...,6,[6]
2,3,Im waiting for someone to say to me that all t...,3 4,"[3, 4]"
3,4,He is a liar. Proven day night. Time again. Li...,6,[6]
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8,[8]


In [9]:
onehot = np.zeros((5000,11))
for i in range(5000) : # 0~5000 row
    for j in df.label[i] : # j = value   
        onehot[i,j]  = 1

In [10]:
all_tweet = np.array(df.Tweet.values)
with open('./all_tweet.txt', 'w', encoding='utf-8') as f:
    for line in all_tweet : 
        if len(str(line).strip()) == 0 : continue
        try : 
            f.write(line + '\n')
        except : continue

In [11]:
!cat all_tweet.txt |head -n10

NO JOKE I WILL HOP ON A PLANE RN! (Well after COVID19 lol)
BanMediaHouse whose is responsible for spreading Fake and communal Stories in this pandemic Corona situation.
Im waiting for someone to say to me that all this corona thing is just an April fools joke
He is a liar. Proven day night. Time again. Lies when the truth will do. COVID19
NEW: U.S. CoronaVirus death toll reaches 4,000 after nearly 900 new deaths were reported today (BNO News) Covid19 CoronaVirusOutbreak
Coronavirus impact Govt extends I-T deadlines related to Sections 80C, 80D
42,000 people might have died in China from Covid19 but China was underreporting according to sources.
Dear Chinese! Kindly cook your bat thoroughly next time. Regards, COVID19 CoronaVirusUpdates CoronaVirus
This is how the govt of kenya is checking the temperatures of COVID19 and saying it is still low. Mtashangaa sana.
My mental health hasn't suffered at all under the coronavirus quarantine! Ha-ha, April Fools.


In [12]:
!pip install sentencepiece
import sentencepiece as spm

     |████████████████████████████████| 1.2MB 14.6MB/s 


In [13]:
spm.SentencePieceTrainer.train('--input=all_tweet.txt --model_prefix=m --vocab_size=8987')
sp =spm.SentencePieceProcessor()
sp.load('m.model')

True

In [14]:
train_df = df[:4000]
test_df  = df[4000:]
print(train_df.shape, test_df.shape)

(4000, 4) (1000, 4)


In [15]:
train_onehot = onehot[:4000]
test_onehot = onehot[4000:]

In [16]:
print(train_onehot.shape, test_onehot.shape)

(4000, 11) (1000, 11)


In [17]:
train_df['Tweet']

0       NO JOKE I WILL HOP ON A PLANE RN! (Well after ...
1       BanMediaHouse whose is responsible for spreadi...
2       Im waiting for someone to say to me that all t...
3       He is a liar. Proven day night. Time again. Li...
4       NEW: U.S. CoronaVirus death toll reaches 4,000...
                              ...                        
3995    You wouldnt have to worry about getting the co...
3996    i'm gonna go ahead and say it, i fucking hate ...
3997    It tells why kerala has exploding case of coro...
3998    ready for corona to call it a joke haha you al...
3999    is my car loan emi (7/4/2020) dedecated from m...
Name: Tweet, Length: 4000, dtype: object

In [18]:
train_df['bow'] = train_df['Tweet'].apply(lambda x : sp.encode_as_ids(str(x)))
test_df['bow'] = test_df['Tweet'].apply(lambda x : sp.encode_as_ids(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
train_df.head()

,ID,Tweet,Labels,label,bow
0,1,NO JOKE I WILL HOP ON A PLANE RN! (Well after ...,0 10,"[0, 10]","[1535, 1582, 10, 1444, 6555, 1481, 77, 4838, 5..."
1,2,BanMediaHouse whose is responsible for spreadi...,6,[6],"[1921, 4543, 4546, 79, 506, 14, 2075, 148, 19,..."
2,3,Im waiting for someone to say to me that all t...,3 4,"[3, 4]","[76, 182, 8, 19, 245, 6, 82, 6, 37, 28, 49, 21..."
3,4,He is a liar. Proven day night. Time again. Li...,6,[6],"[360, 14, 9, 2977, 3, 4436, 4293, 102, 552, 3,..."
4,5,"NEW: U.S. CoronaVirus death toll reaches 4,000...",8,[8],"[1571, 42, 306, 3, 137, 3, 269, 201, 525, 854,..."


In [20]:
train_text = train_df['bow'].values
test_text = test_df['bow'].values

In [21]:
train_text.shape, test_text.shape

((4000,), (1000,))

In [22]:
train_text[:2]

array([list([1535, 1582, 10, 1444, 6555, 1481, 77, 4838, 5224, 46, 220, 4549, 193, 20, 17, 369, 160]),
       list([1921, 4543, 4546, 79, 506, 14, 2075, 148, 19, 443, 2883, 15, 2588, 157, 8420, 18, 21, 155, 24, 581, 3])],
      dtype=object)

In [23]:
train_bow_text = tf.keras.preprocessing.sequence.pad_sequences(train_text)
test_bow_text = tf.keras.preprocessing.sequence.pad_sequences(test_text)

In [24]:
train_bow_text[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0, 1535, 1582,   10, 1444, 6555,
       1481,   77, 4838, 5224,   46,  220, 4549,  193,   20,   17,  369,
        160], dtype=int32)

In [25]:
train_bow_text.shape

(4000, 45)

In [26]:
test_bow_text.shape

(1000, 44)

In [27]:
import collections 
word_counter = collections.Counter()

for text in train_text :
    word_counter.update(text)
for text in test_text :
    word_counter.update(text)


In [28]:
def n_cut(texts, n) :
    return np.array([[word for word in text if word_counter[word] >= n ] for text in texts])

In [29]:
import warnings
warnings.filterwarnings('ignore')

In [30]:
train_cut_text = n_cut(train_text,10)
test_cut_text = n_cut(test_text,10)

train_cut_text.shape, test_cut_text.shape

((4000,), (1000,))

In [31]:
train_cut_bow_text = tf.keras.preprocessing.sequence.pad_sequences(train_cut_text, maxlen=80)
test_cut_bow_text = tf.keras.preprocessing.sequence.pad_sequences(test_cut_text, maxlen=80)

In [32]:
train_cut_bow_text.shape, test_cut_bow_text.shape

((4000, 80), (1000, 80))

In [33]:
train_onehot.shape, test_onehot.shape

((4000, 11), (1000, 11))

In [34]:
# bow -> text
def bow2txt(bow) :
    return sp.decode_ids(bow)

In [35]:
bow2txt(test_text[2])

'On the Coronavirus, the dock media is doing Hindu-Muslim, you should make a genuine news on it.'

## MODELING

In [64]:
from keras.layers import Input, Dense, GRU, Embedding, SimpleRNN
from keras.models import Model
def seq2seq() :
    input_x_bow = Input(shape=(80,))
    embedding = Embedding(8987,160)
    x = embedding(input_x_bow)
    l1 = SimpleRNN(100, return_sequences = True)(x)
    l2 = SimpleRNN(100)(l1)
    #z = GRU(128)(x)
    #z = GRU(64)(l2)
    #z = GRU(64)(z)
    #z = GRU(64)(z)


    y = Dense(11, activation = 'sigmoid')(l2)

    model = Model(input_x_bow, y)
    model.compile(
        loss = 'binary_crossentropy',
        optimizer = 'adam',
        metrics = ['accuracy']
    )

    return model

In [65]:
model = seq2seq()
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 80)]              0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 80, 160)           1437920   
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 80, 100)           26100     
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 100)               20100     
_________________________________________________________________
dense_7 (Dense)              (None, 11)                1111      
Total params: 1,485,231
Trainable params: 1,485,231
Non-trainable params: 0
_________________________________________________________________


In [66]:
hist = model.fit(train_cut_bow_text, train_onehot,
                 validation_data = (test_cut_bow_text, test_onehot),
                 epochs = 20, 
                 verbose = 1)

Epoch 1/20
125/125 [==============================] - 20s 112ms/step - loss: 0.4681 - accuracy: 0.1043 - val_loss: 0.4220 - val_accuracy: 0.0970
Epoch 2/20
111/125 [=========================>....] - ETA: 1s - loss: 0.4102 - accuracy: 0.1169

KeyboardInterrupt: ignored